#### Demostración de info para hallar ángulo de fase de la fuente 

In [ ]:
with open("angle_test.atp", "r+") as f:
    lines = f.readlines()
lines_copy = lines.copy()
for line_idx, line in enumerate(lines):
    if "C MainSource" == line.strip("\n"):
        print(line_idx, line)
        print(lines[line_idx+1][0:8])
        print(lines[line_idx+1][8:16])
        print(f"16:24: {lines[line_idx+1][16:24]}")
        print(f"24:32: {lines[line_idx+1][24:32]}")
        print(f"32:40: {lines[line_idx+1][32:40]}")
        print(f"40:48: {lines[line_idx+1][40:48]}")
        print(f"48:56: {lines[line_idx+1][48:56]}")
        print(f"56:64: {lines[line_idx+1][56:64]}")
        print(f"64:72: {lines[line_idx+1][64:72]}")
        print(f"64:72: {lines[line_idx+1][64:72]}")
        print(f"72:80: {lines[line_idx+1][72:80]}")

        print(f"Ángulo de fase utiliza 3 digitos y 5 decimales (9 posiciones):")
        print(f"En las columnas [31:40]: {lines[line_idx+1][31:40]}")
        source_phase = lines[line_idx+1][31:40]

# print("C MainSource\n" in lines_copy)

In [125]:
print(source_phase)
import random

phase_int = random.randint(0, 360)
phase_dec = random.random()
source_phase = phase_int + phase_dec

print(source_phase)
if random_phase:
    print("true")


118.64052626406269
188.84599671403654


In [127]:
with open("angle_test.atp") as f:
    lines = f.readlines()
lines_copy = lines.copy()
random_phase = True
for line_idx, line in enumerate(lines):
    if "C MainSource" in line:
        d_mainSource = line_idx + 1
phase_line = lines_copy[d_mainSource]
if random_phase:
    lines_copy[d_mainSource] = f"{phase_line[:30]}{str(round(source_phase, 5)):>10}{phase_line[40:]}"
    lines_copy[d_mainSource+1] = f"{phase_line[:30]}{str(round(source_phase-120,5)):>10}{phase_line[40:]}"
    lines_copy[d_mainSource+2] = f"{phase_line[:30]}{str(round(source_phase-240,5)):>10}{phase_line[40:]}"
print(phase_line)
print(lines_copy[d_mainSource])
with open("archivoprueba.atp", "w") as f:
    f.writelines(lines_copy)

14SRCA    8164.96581       60. 123.12345                           -1.      100.

14SRCA    8164.96581       60.   188.846                           -1.      100.



In [128]:
try:
    lines_copy[alvaro]
except NameError:
    print("Check mainSource/microGridSource comment")



Check mainSource/microGridSource comment
Alvaro


In [ ]:
with open("IEEE34_Base.atp", "r+") as f:
    lines = f.readlines()
lines_copy = lines.copy()
for line_idx, line in enumerate(lines):
    if "C MainSource" == line.strip("\n"):
        print(line_idx, line)
        print(lines[line_idx+1][0:8])
        print(lines[line_idx+1][8:16])
        print(f"16:24: {lines[line_idx+1][16:24]}")
        print(f"24:32: {lines[line_idx+1][24:32]}")
        print(f"32:40: {lines[line_idx+1][32:40]}")
        print(f"40:48: {lines[line_idx+1][40:48]}")
        print(f"48:56: {lines[line_idx+1][48:56]}")
        print(f"56:64: {lines[line_idx+1][56:64]}")
        print(f"64:72: {lines[line_idx+1][64:72]}")
        print(f"64:72: {lines[line_idx+1][64:72]}")
        print(f"72:80: {lines[line_idx+1][72:80]}")

        print(f"Ángulo de fase utiliza 3 digitos y 5 decimales (9 posiciones):")
        print(f"En las columnas [31:40]: {lines[line_idx+1][31:40]}")